# Análisis numérico
## Discusión 3 :  solución

## Método de bisección

$\textbf{1.a.}$ Para transformar la ecuación en un problema de encontrar raíces, simplemente buscamos que quede igualada a $0$ :

$$ f(c_{f}) = 1.74 \ln ( Re \sqrt{c_{f}} ) - 0.4 - \sqrt{\frac{1}{c_{f}}} = 0 $$

Para poder aplicar el método de bisección necesitamos que la función sea continua en el intervalo dado $[0.005, 0.01]$ y que tome valores de signos opuestos en ambos extremos. La continuidad la podemos deducir del hecho que la función está compuesta por funciones que son continuas en el intervalo. Para verificar los signos opuestos, definiremos la función.

### Dependencias

In [14]:
from math import log, ceil, sqrt

### Parámetros

In [15]:
# Dados en el problema
a = 0.005
b = 0.01
Re = 10**4

# Definimos la función anterior
def f(x):
    return 1.74*log(Re*sqrt(x)) - 0.4 - sqrt(1/x)

Ahora podemos verificar que la función cumple la segunda condición:

In [16]:
print(f(a)) # Extremo izquierdo del intervalo
print(f(b)) # Extremo derecho del intervalo

-3.1256794853891865
1.619494185428918


¿Qué habría pasado si una de estas condiciones no se hubiese cumplido? 
No podríamos garantizar la convergencia del método.

Ahora vamos a aniquilar el $\textbf{1.b.}$ y el $\textbf{1.c.}$ de una sola vez: vamos a hacer que el algoritmo calcule el número de iteraciones, según la fórmula presentada en discusión, para alcanzar una tolerancia deseada.

In [17]:
'''
f: función para la cual buscamos la raíz 
a: extremo izquierdo del intervalo
b: extremo derecho del intervalo
T: tolerancia 
'''

def bisection(f, a, b, T):
    n = ceil( log( (b - a)/T  , 2) ) #En esta línea estoy calculando el número de iteraciones necesarias
    for i in range(0, n):
        p = (a + b)/2
        print(str(i) + " \t " + str(a) + " \t " + str(p) + " \t " + str(b) + " \t " + str(f(p)) )
        if f(a)*f(p) < 0:
            b = p
        else:
            a = p

Ahora aplicamos el esquema anterior a la función $f$ que definimos arriba, pasándole una tolerancia de $10^{-6}$.

In [10]:
bisection(f, a, b, 10**-6)

0 	 0.005 	 0.0075 	 0.01 	 -0.1777946013966467
1 	 0.0075 	 0.00875 	 0.01 	 0.8128721973486073
2 	 0.0075 	 0.008125 	 0.00875 	 0.344844013567263
3 	 0.0075 	 0.0078125 	 0.008125 	 0.09101741864372848
4 	 0.0075 	 0.00765625 	 0.0078125 	 -0.041421866309180544
5 	 0.00765625 	 0.007734375 	 0.0078125 	 0.025277253136721356
6 	 0.00765625 	 0.0076953125 	 0.007734375 	 -0.007950945331794301
7 	 0.0076953125 	 0.00771484375 	 0.007734375 	 0.00869330567140203
8 	 0.0076953125 	 0.0077050781250000006 	 0.00771484375 	 0.0003787415508345049
9 	 0.0076953125 	 0.0077001953125 	 0.0077050781250000006 	 -0.0037842086041166567
10 	 0.0077001953125 	 0.00770263671875 	 0.0077050781250000006 	 -0.001702260573193115
11 	 0.00770263671875 	 0.007703857421875 	 0.0077050781250000006 	 -0.0006616413187927606
12 	 0.007703857421875 	 0.0077044677734375 	 0.0077050781250000006 	 -0.0001414203416256754


Ese es un resultado bastante chueco: si queremos mejores resultados debemos poner una tolerancia más pequeña, lo que aumentará el número de iteraciones.

## Método de punto fijo

$\textbf{2.a.}$ Para encontrar la función $g$ deseada, debemos dejar la variable, $c_{f}$, sola en un lado de la ecuación. Luego de un poco de álgebra podemos llegar a 

$$ c_{f}  = \frac{1}{(-0.4 + 1.74 \ln (Re \sqrt{c_{f}}) )^{2}}. $$ 

Vamos a definir $g(c_{f})$ igual al lado derecho

$$ g(c_{f}) = \frac{1}{(-0.4 + 1.74 \ln (Re \sqrt{c_{f}}) )^{2}} $$

y vamos a rogar que esta función cumpla las condiciones necesarias para tener un esquema de punto fijo convergente. Primero definámosla:

In [21]:
def g(x):
    return 1/(1.74*log(Re*sqrt(x)) - 0.4)**2

Verifiquemos condiciones:

$\textbf{Existencia. }$ En discusión les comenté que va a ser suficiente que verifiquen que $g(a) \in [a, b]$ y $g(b) \in [a, b]$. Veamos...

In [22]:
print(g(a))
print(g(b))

0.008239790707770357
0.007406713638360005


Ambos valores están dentro del intervalo de interés, que es $[0.005, 0.01]$. Concluimos que existe un punto fijo.

$\textbf{Unicidad. }$ Al evaluar con la calculadora $\texttt{fmax}( \mid g'(x) \mid, x, 0.005, 0.01)$ obtenemos que 

$$ \max_{0.005 \leq x \leq 0.01} \mid g'(x) \mid = \mid g'(0.005) \mid = 0.260288 $$

Esta es una constante positiva menor que $1$, por lo que concluimos que el punto fijo es único. Ahora programemos el esquema.

In [30]:
'''
f: función para la cual buscamos el punto fijo 
a: extremo izquierdo del intervalo
b: extremo derecho del intervalo
p: aproximación inicial
k: valor máximo del valor absoluto de la primera derivada en el intervalo
T: tolerancia
'''

def fixedPoint(g, a, b, p, k, T):
    #En la sig. línea estoy calculando el número de iteraciones necesarias :)
    n = ceil( log( T/( max(b - p, p - a) ) , k))  
    for i in range(0, n):
        print(str(i) + " \t " + str(p))
        p = g(p)

$\textbf{2.b.}$ y $\textbf{2.c.}$ Vamos a iterar usando como punto inicial el punto al medio del intervalo y una tolerancia de $10^{-6}$.

In [31]:
fixedPoint(g, a, b, (a + b)/2, 0.260288, 10**-6)

0 	 0.0075
1 	 0.007736407966250012
2 	 0.007699793076400809
3 	 0.007705373304155738
4 	 0.0077045207552921754
5 	 0.0077046509588405135


Ahora una tolerancia de $10^{-9}$.

In [32]:
fixedPoint(g, a, b, (a + b)/2, 0.260288,  10**-9)

0 	 0.0075
1 	 0.007736407966250012
2 	 0.007699793076400809
3 	 0.007705373304155738
4 	 0.0077045207552921754
5 	 0.0077046509588405135
6 	 0.00770463107265981
7 	 0.007704634109878903
8 	 0.007704633646003393
9 	 0.007704633716851246
10 	 0.007704633706030629
